Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [1]:
from typing import Callable
from itertools import product
from random import random, randint, shuffle, seed
import numpy as np
from nptyping import NDArray, Shape, Int, Bool
from scipy import sparse as sp
from math import ceil
from pprint import pprint

In [2]:
seed_initializer = 2654435761

def make_set_covering_problem(num_points, num_sets, density) -> sp.lil_array:
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points * seed_initializer + num_sets + density)
    sets = sp.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets

# Halloween Challenge

Find the best solution with the fewest calls to the fitness functions for:

* `num_points = [100, 1_000, 5_000]`
* `num_sets = num_points`
* `density = [.3, .7]` 

In [3]:
problem_sizes = [100, 1_000, 5_000]
densities = [.3, .7]

In [4]:
def overlap_sets(covering_sets: sp.lil_array) -> NDArray[Shape["1, ..."], Bool]:
    sum: NDArray[Shape["1, ..."], Int] = covering_sets.sum(axis=0) # dtype=bool?
    boolean_array_mapper = np.vectorize(lambda x: x > 0)
    return boolean_array_mapper(sum)

In [5]:
def fitness(state: list[bool], covering_sets: sp.lil_array):
    taken_sets_indices: list[int] = [i for i, x in enumerate(state) if x]
    # taken_sets: sparse.lil_array = filter_covering_sets(covering_sets, taken_sets_indices)
    taken_sets: sparse.lil_array = covering_sets[taken_sets_indices, :] # type: ignore
    reduced = overlap_sets(taken_sets)
    coverage = sum(reduced)
    cost = len(taken_sets_indices)
    return (coverage, -cost)

In [6]:
def tweak(state: list[bool], probability_weights: NDArray | None = None) -> list[list[bool]]:
    ''' Tweak may be fed with an array of probability weights, which purpose is to increase the probability
        that the tweak is made on a covering set with a rare item '''
    if probability_weights is not None:
        changing_index = np.random.choice(len(state), p=probability_weights)
    else:
        changing_index = randint(0, len(state) - 1)
    return [[not x if i == changing_index else x for i, x in enumerate(state)]]

In [7]:
def elements_rarity_scores(covering_sets: sp.lil_array) -> NDArray[Shape["1, ..."], Int]:
    rows_sum: NDArray[Shape["1, ..."], Int] = covering_sets.sum(axis=0) # shape is [1, number of points]
    sets_per_element: int = ceil(np.average(rows_sum).item()) # approximated
    elements_rarity_score: NDArray = np.repeat(sets_per_element, len(rows_sum)) - rows_sum
    return np.where(elements_rarity_score >= 0, elements_rarity_score, np.zeros(shape = elements_rarity_score.shape, dtype = elements_rarity_score.dtype))


In [8]:
# https://stackoverflow.com/questions/77417902/how-to-map-scipy-sparse-lil-array
def covering_sets_scores(covering_sets: sp.lil_array, elements_scores: NDArray[Shape["1, ..."], Int]) -> NDArray[Shape["1, ..."], Int]:
    assert len(elements_scores.shape) == 1, "x should be 1d"
    row, col = covering_sets.nonzero()
    xx = elements_scores[col]
    data = np.empty(row.shape, dtype=elements_scores.dtype)
    data[:] = xx
    sets_with_scores = sp.coo_matrix((data, (row, col)), shape=covering_sets.shape)
    sets_with_scores.eliminate_zeros()
    lil: sp.lil_matrix = sets_with_scores.tolil()
    set_scores: NDArray = np.asarray(lil.sum(axis=1))
    return np.squeeze(set_scores)


In [9]:
def covering_sets_weights(covering_sets: sp.lil_array):
    ''' Returns the average of a uniform distribution and normalized set_scores '''
    elements_scores = elements_rarity_scores(covering_sets)
    set_scores = covering_sets_scores(covering_sets, elements_scores)
    norm_one = sum(set_scores)
    return (np.repeat(1 / len(set_scores), len(set_scores)) + set_scores / norm_one) / 2

In [10]:
def solve(problem_size: int, density: float, fitness_func: Callable, tweak_func: Callable, computing_set_scores = False, step_limit = 200):
    covering_sets: sp.lil_array = make_set_covering_problem(num_points=problem_size, num_sets=problem_size, density=density)
    initial_state = [False for _ in range(problem_size)]
    probability_weights = covering_sets_weights(covering_sets) if computing_set_scores else None
    fitness_calls = 0

    step = 1
    non_improving_steps = 0
    current_fitness, fitness_calls = fitness_func(initial_state, covering_sets), fitness_calls + 1
    current_state = initial_state
    while step <= step_limit:
        children_states = tweak(current_state, probability_weights)
        for child_state in children_states:
            child_fitness, fitness_calls = fitness_func(child_state, covering_sets), fitness_calls + 1
            if child_fitness > current_fitness:
                current_fitness, current_state = child_fitness, child_state
                non_improving_steps = 0
        if non_improving_steps > 5:
            break
        step += 1
        non_improving_steps += 1

    return (current_fitness, fitness_calls)


In [11]:
for problem_size, density in product(problem_sizes, densities):
    fitness_score, calls = solve(problem_size, density, fitness, tweak, computing_set_scores = True)
    print(fitness_score, calls, 'Using weights')
    fitness_score, calls = solve(problem_size, density, fitness, tweak, computing_set_scores = False)
    print(fitness_score, calls)

(100, -11) 20 Using weights
(100, -10) 21
(100, -4) 11 Using weights
(100, -4) 11
(999, -16) 26 Using weights
(1000, -19) 29
(1000, -6) 13 Using weights
(1000, -6) 13
(5000, -23) 35 Using weights
(5000, -22) 31
(5000, -8) 18 Using weights
(5000, -8) 16
